In [ ]:
from ParticleSystem import ParticleSystem

WORLD_WIDTH = 800
WORLD_HEIGHT = 600
NUM_PARTICLES = 300
DELTA = 0.5
MU = 0.5



world = ParticleSystem(
    NUM_PARTICLES,
    WORLD_WIDTH,
    WORLD_HEIGHT,
    DELTA,
    MU
)

for p in world.particles:
    print(f'({p.x} {p.y})')


# world.run_simulation()


(586 67)
(49 526)
(779 322)
(301 160)
(572 431)
(310 417)
(159 11)
(578 414)
(316 555)
(405 373)
(446 261)
(524 456)
(350 441)
(475 590)
(439 526)
(296 391)
(378 516)
(673 322)
(258 254)
(782 452)
(518 323)
(781 357)
(339 343)
(797 332)
(56 28)
(519 505)
(300 430)
(102 43)
(658 80)
(127 267)
(130 81)
(360 293)
(461 340)
(686 162)
(360 87)
(497 252)
(162 554)
(151 411)
(419 67)
(498 574)
(377 25)
(1 97)
(382 192)
(523 544)
(567 459)
(303 72)
(571 444)
(71 205)
(248 422)
(254 535)
(600 435)
(610 221)
(599 248)
(459 396)
(611 522)
(567 582)
(580 561)
(370 89)
(211 566)
(303 365)
(616 190)
(619 450)
(288 461)
(670 61)
(121 593)
(376 531)
(303 461)
(347 448)
(161 227)
(224 307)
(562 431)
(348 52)
(381 448)
(190 80)
(705 595)
(161 319)
(518 251)
(154 502)
(763 482)
(526 217)
(176 387)
(20 357)
(430 538)
(755 518)
(642 303)
(747 344)
(330 250)
(713 480)
(585 340)
(544 90)
(53 379)
(592 553)
(775 358)
(164 133)
(400 409)
(406 14)
(246 109)
(656 395)
(472 53)
(115 244)
(302 378)
(612 11)
(249 4

In [6]:
from PIL import Image

# Create a new 1x1 image with a white dot
image = Image.new("RGB", (1, 1), color="white")

# Save the image as 'white_dot.png'
image.save("white_dot.png")

print("1x1 white dot image created and saved as white_dot.png")

1x1 white dot image created and saved as white_dot.png


: 